In [27]:
from pathlib import Path

import pandas as pd
import requests
import tomli

In [2]:
API_TOML_DIR = Path(Path.cwd().parent, "api_creds.toml")

In [3]:
with open(API_TOML_DIR, 'rb') as f:
    key = tomli.load(f)['api_key']

In [4]:
def get_forecast(lat_lon: tuple[float, float], forecast_days:int = 3, api_key: str = key) -> dict:
    """Gets a forecase for a given lat lon from the https://www.weatherapi.com/ site. 
    Requires an api key to be defined. 

    Args:
        lat_lon (tuple[float, float]): lat/lon of the location to be forecasted
        forecast_days (int, optional): number of days to forecast, note free tier weatherapi is restricted to 14 days. Defaults to 3.
        api_key (str, optional): api key for weather api. Defaults to key.

    Raises:
        SystemExit: generic error catch for incorrect request parameters

    Returns:
        dict: json of the returned api call#
    """
    BASE_URL = "https://api.weatherapi.com/v1/forecast.json?"
    str_lat_lon = ",".join([str(x) for x in lat_lon])
    query_params = {
    'q': str_lat_lon,
    'days': forecast_days, 
    'key':api_key}
    try:
        response = requests.get(BASE_URL, params=query_params)
    except requests.exceptions.RequestException as e:  # TODO Generic error catching = bad
        raise SystemExit(e)
    return response.json()

In [25]:
def parse_forcast_response(forecast_response_json:dict) -> dict:
    num_days_forecasted = len(forecast_response_json['forecast']['forecastday'])
    lat = forecast_response_json['location']['lat']
    lon = forecast_response_json['location']['lon']
    dates = []
    max_temps = []
    min_temps = []
    for day in range(num_days_forecasted):
        dates.append(forecast_response_json['forecast']['forecastday'][day]['date'])
        max_temps.append(forecast_response_json['forecast']['forecastday'][day]['day']['maxtemp_c'])
        min_temps.append(forecast_response_json['forecast']['forecastday'][day]['day']['mintemp_c'])
    return {'lat':lat, 'lon':lon, 'dates':dates, 'max_temps':max_temps, 'min_temps':min_temps}

In [30]:
response = parse_forcast_response(get_forecast(lat_lon = (48.5, 2.35)))

In [32]:
pd.DataFrame.from_dict(response)

,lat,lon,dates,max_temps,min_temps
0,48.5,2.35,2023-07-18,34.5,13.0
1,48.5,2.35,2023-07-19,28.7,15.2
2,48.5,2.35,2023-07-20,27.2,13.7
